In [1]:
import openmc
import openmc.deplete
import numpy as np

In [2]:
# Material
fuel = openmc.Material(name='fuel')# 建立燃料对象
fuel.set_density(units="atom/b-cm",density=8.0175E-2)# 设置燃料密度
fuel.add_nuclide("U233",1.5621E-3,percent_type="ao")# 添加核素U-233，含量占比为原子数比例
fuel.add_nuclide("Th232",1.2639E-2,percent_type="ao")# 添加核素Th-232，含量占比为原子数比例
fuel.add_nuclide("O16",2.8402E-2,percent_type="ao")# 添加核素O-16，含量占比为原子数比例
fuel.temperature = 1500 # 设置燃料温度，单位为Kelvin

############Depletion############
fuel.depletable = True
fuel.volume = np.pi*0.52273**2
############Depletion############

clad = openmc.Material(name='clad')
clad.set_density(units='atom/b-cm',density=4.3346E-2)
clad.add_nuclide("Zr91",1,percent_type="ao")
clad.temperature = 600

moderator = openmc.Material(name="moderator")
moderator.set_density(units="atom/b-cm",density=6.5491E-2)
moderator.add_nuclide("H2",4.3661E-2,percent_type="ao")
moderator.add_nuclide("O16",2.1830E-2,percent_type="ao")
moderator.temperature = 600
moderator.add_s_alpha_beta('c_D_in_D2O') # 添加水中氘的热散射截面

materials = openmc.Materials([fuel, clad, moderator])

In [3]:
# Geometry parameters
fuel_r = [0.52273,0.57273] # 燃料栅元半径 （忽略气隙）
pitch = 1.26 # 栅距

In [4]:
# Pin universe
pin_univ = openmc.Universe(name='pin')

# Surface
fuel_outer = openmc.ZCylinder(r=fuel_r[0])
clad_outer = openmc.ZCylinder(r=fuel_r[1])
bound_box = openmc.rectangular_prism(pitch, pitch, boundary_type="reflective")

# Cell
fuel_cell = openmc.Cell(name='fuel cell')
fuel_cell.fill = fuel
fuel_cell.region = -fuel_outer
pin_univ.add_cell(fuel_cell)

clad_cell = openmc.Cell(name='clad cell')
clad_cell.fill = clad
clad_cell.region = +fuel_outer & -clad_outer
pin_univ.add_cell(clad_cell)

mod_cell = openmc.Cell(name='moderator cell')
mod_cell.fill = moderator
mod_cell.region = bound_box
mod_cell.region = mod_cell.region & +clad_outer
pin_univ.add_cell(mod_cell)

In [5]:
geometry = openmc.Geometry(pin_univ)

In [6]:
# 设置计算参数
# settings
settings = openmc.Settings() # 创建参数对象
settings.particles = 10000 # 粒子数
settings.inactive = 100 # 非活跃代
settings.batches = 200 # 总代数
settings.temperature['multipole']= True # 调用Windowed Multipole做多普勒展开
settings.temperature['method']= 'interpolation' 

In [7]:
# 绘制几何结构
# plot
plot_xy = openmc.Plot(plot_id=1)
plot_xy.basis = 'xy'
plot_xy.filename = 'materials-xy-Height'
plot_xy.origin = [0, 0,0]
plot_xy.pixels = [3000, 3000]
plot_xy.width = (2, 2)
plot_xy.color_by = 'material'

In [9]:
# 输出上述几何、计算设置、材料、几何绘制文件
# export to xml
geometry.export_to_xml()
settings.export_to_xml()
materials.export_to_xml()
plot_file = openmc.Plots([plot_xy])
plot_file.export_to_xml()

In [10]:
# 绘制几何
openmc.plot_geometry(output=False)

In [ ]:
############Depletion############

In [11]:
chain = openmc.deplete.Chain.from_xml("./chain_casl_pwr.xml")

In [12]:
# 如果geometry和settings这两个变量仍在内存里，可以直接调用；否则，通过下述函数读取
# geometry = openmc.Geometry.from_xml()
# settings = openmc.Settings.from_xml()

/home/bearsanxw/miniconda3/lib/python3.7/site-packages/openmc/mixin.py:68: IDWarning: Another Surface instance already exists with id=1.
  warn(msg, IDWarning)
/home/bearsanxw/miniconda3/lib/python3.7/site-packages/openmc/mixin.py:68: IDWarning: Another Surface instance already exists with id=2.
  warn(msg, IDWarning)
/home/bearsanxw/miniconda3/lib/python3.7/site-packages/openmc/mixin.py:68: IDWarning: Another Surface instance already exists with id=3.
  warn(msg, IDWarning)
/home/bearsanxw/miniconda3/lib/python3.7/site-packages/openmc/mixin.py:68: IDWarning: Another Surface instance already exists with id=4.
  warn(msg, IDWarning)
/home/bearsanxw/miniconda3/lib/python3.7/site-packages/openmc/mixin.py:68: IDWarning: Another Surface instance already exists with id=5.
  warn(msg, IDWarning)
/home/bearsanxw/miniconda3/lib/python3.7/site-packages/openmc/mixin.py:68: IDWarning: Another Surface instance already exists with id=6.
  warn(msg, IDWarning)
/home/bearsanxw/miniconda3/lib/python3.7

In [14]:
# 设置燃耗
# 几何、计算设置、燃耗链
operator = openmc.deplete.Operator(geometry, settings, "./chain_casl_pwr.xml")

In [15]:
# 给定总功率，单位为W。如果是二维问题，则是W/cm
power = 174
# 通过列表给定燃耗计算时间步，默认单位是s
time_steps = [30 * 24 * 60 * 60] * 6

In [ ]:
# 设置燃耗求解器
# 选用 PredictorIntegrator
integrator = openmc.deplete.PredictorIntegrator(operator, time_steps, power)

In [ ]:
# 启动燃耗计算
integrator.integrate()